In [1]:
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display
import random
import warnings
import os
from PIL import Image
import pathlib
import csv
# sklearn Preprocessing
from sklearn.model_selection import train_test_split
#Keras
import keras
import warnings
warnings.filterwarnings('ignore')
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD


Collab için drive işlemleri

In [2]:
#drive tanımlaması yapılmak zorunda
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

 Her ses işlenip spectrogramları png olarak kaydediyor.

In [4]:

words=['down', 'eat', 'sleep', 'up']

current_path = "drive/My Drive/training_sets/"
 
for i in range (len(words)):
  pathlib.Path(f'img_data2/{words[i]}').mkdir(parents=True, exist_ok=True) #for images                           
  for j in range (10):
    path = current_path +  words[i] + "-" + str(j+1) + ".wav"  
    y, sr = librosa.load(path, mono=True, duration=5)
    plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, sides='default', mode='default', scale='dB');
    plt.axis('off');
    plt.savefig((f'img_data2/{words[i]}/{words[i]}-{j+1}.png'))    #Kaydet
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [5]:
!pip install split-folders

In [6]:
#For  dataset
import splitfolders 

splitfolders.ratio('./img_data2/', output="./data2", seed=1337, ratio=(.8, .2)) # default values
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator( #(X_train , y_train return)
        rescale=1./255,       # all our pixel values are in range (0,1)
        shear_range=0.2,      #to apply some random tranfromations
        zoom_range=0.2,       
        horizontal_flip=True) 

test_datagen = ImageDataGenerator( #(X_test , y_test return)
        rescale=1./255,       # all our pixel values are in range (0,1)
        shear_range=0.2,      #to apply some random tranfromations
        zoom_range=0.2,      
        horizontal_flip=True) 

Copying files: 40 files [00:00, 5219.56 files/s]


In [7]:
#data generators

#class-mode -> yapılacak sınıflandırmaya göre uygun olanı seçilmelidir. (binary - categorical)     
training_set = train_datagen.flow_from_directory( 
    './data2/train',
    target_size=(64, 64),
    batch_size=1,
    class_mode='binary',
    shuffle = False)

test_set = test_datagen.flow_from_directory(
    './data2/val',
    target_size=(64, 64),
    batch_size=1,
    class_mode='binary',
    shuffle = False )



Found 32 images belonging to 4 classes.
Found 8 images belonging to 4 classes.


In [8]:
model = Sequential()
input_shape=(64, 64, 3)                                              #1st hidden layer
model.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))                                         #2nd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))                                         #3rd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))                                         #Flatten
model.add(Flatten())
model.add(Dropout(rate=0.5))                                          #Add fully connected layer.
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))                                          #Output layer
model.add(Dense(10))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 31, 31, 32)        896       
_________________________________________________________________
average_pooling2d (AveragePo (None, 15, 15, 32)        0         
_________________________________________________________________
activation (Activation)      (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
average_pooling2d_1 (Average (None, 7, 7, 64)          0         
_________________________________________________________________
activation_1 (Activation)    (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          3

In [9]:
epochs = 200
batch_size = 8
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer="sgd",  loss ="sparse_categorical_crossentropy", metrics=['accuracy'])

In [17]:
model.fit_generator(
        training_set,
        steps_per_epoch= len(training_set), 
        epochs=200,
        validation_data=test_set,
        validation_steps= len(test_set), 
         )

Epoch 1/200
32/32 [==============================] - 0s 10ms/step - loss: 1.3358 - accuracy: 0.2812 - val_loss: 1.2137 - val_accuracy: 0.3750
Epoch 2/200
32/32 [==============================] - 0s 8ms/step - loss: 1.3993 - accuracy: 0.4375 - val_loss: 1.2507 - val_accuracy: 0.2500
Epoch 3/200
32/32 [==============================] - 0s 9ms/step - loss: 1.3068 - accuracy: 0.4062 - val_loss: 1.1554 - val_accuracy: 0.3750
Epoch 4/200
32/32 [==============================] - 0s 8ms/step - loss: 1.3012 - accuracy: 0.4375 - val_loss: 1.1454 - val_accuracy: 0.5000
Epoch 5/200
32/32 [==============================] - 0s 8ms/step - loss: 1.3086 - accuracy: 0.4062 - val_loss: 1.0844 - val_accuracy: 0.5000
Epoch 6/200
32/32 [==============================] - 0s 9ms/step - loss: 1.2550 - accuracy: 0.3750 - val_loss: 1.1118 - val_accuracy: 0.5000
Epoch 7/200
32/32 [==============================] - 0s 8ms/step - loss: 1.1736 - accuracy: 0.3750 - val_loss: 1.0435 - val_accuracy: 0.5000
Epoch 8/200


In [19]:
#Model Evaluation
model.evaluate_generator(generator=test_set, steps=200)

[0.3373205065727234, 0.875]

In [20]:
#test veri setini değiştiriyoruz. Ama önce sıfırlanması gerekiyor
test_set.reset()
pred = model.predict_generator(test_set, verbose=1) #steps= len(test_set)

8/8 [==============================] - 0s 5ms/step


In [21]:
predicted_class_indices=np.argmax(pred,axis=1)
#print(predicted_class_indices)

labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:600]
filenames=test_set.filenames
#print(filenames)

In [22]:
print(len(filenames), len(predictions))


8 8


Karşılaştırmalar için tablo oluşturuldu.

In [23]:
import csv
results=pd.DataFrame({"Filename" :filenames,
                      "Predictions":predictions})

    
results.to_csv("prediction_results.csv",index=False)